In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

C:\Users\adhva\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
url = 'https://docs.google.com/spreadsheets/d/1_y0_LJmSY6sNx8qd51T70n0oa_ugN50AVFKuJmXO1-s/export?format=csv&gid=2042605142'
approval = pd.read_csv(url)
approval

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,methodology,state,start_date,end_date,sponsor_candidate_id,...,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,dem,rep,ind
0,89624.0,1102.0,Emerson,NaN,NaN,Online Opt-In Panel/Email,NaN,3/2/25,3/3/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,44.4,41.4,NaN
1,89604.0,1554.0,RMG Research,2178.0,Napolitan News Service,NaN,NaN,2/20/25,2/21/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,46.0,48.0,NaN
2,89579.0,568.0,YouGov,352.0,Economist,Online Matched Sample,NaN,2/16/25,2/18/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,35.0,37.0,NaN
3,89579.0,568.0,YouGov,352.0,Economist,Online Matched Sample,NaN,2/16/25,2/18/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,42.0,41.0,NaN
4,89590.0,294.0,McLaughlin,NaN,NaN,Online Opt-In Panel,NaN,2/11/25,2/18/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,41.0,48.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,NaN,NaN,NaN,NaN
115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,NaN,NaN,NaN,NaN
116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,NaN,NaN,NaN,NaN
117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,NaN,NaN,NaN,NaN


In [4]:
approval = approval[~approval['poll_id'].isna()]
approval

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,methodology,state,start_date,end_date,sponsor_candidate_id,...,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,dem,rep,ind
0,89624.0,1102.0,Emerson,NaN,NaN,Online Opt-In Panel/Email,NaN,3/2/25,3/3/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,44.4,41.4,NaN
1,89604.0,1554.0,RMG Research,2178.0,Napolitan News Service,NaN,NaN,2/20/25,2/21/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,46.0,48.0,NaN
2,89579.0,568.0,YouGov,352.0,Economist,Online Matched Sample,NaN,2/16/25,2/18/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,35.0,37.0,NaN
3,89579.0,568.0,YouGov,352.0,Economist,Online Matched Sample,NaN,2/16/25,2/18/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,42.0,41.0,NaN
4,89590.0,294.0,McLaughlin,NaN,NaN,Online Opt-In Panel,NaN,2/11/25,2/18/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,41.0,48.0,NaN
5,89596.0,1497.0,co/efficient,NaN,NaN,NaN,NaN,2/15/25,2/17/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,44.0,46.0,NaN
6,89566.0,1302.0,Echelon Insights,NaN,NaN,Online Opt-In Panel,NaN,2/10/25,2/13/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,46.0,47.0,NaN
7,89565.0,1895.0,Quantus Insights,2184.0,TrendingPolitics,Online Opt-In Panel,NaN,2/10/25,2/12/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,44.0,48.0,NaN
8,89553.0,921.0,Clarity,NaN,NaN,Online Opt-In Panel,NaN,1/31/25,2/6/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,45.0,44.0,NaN
9,89560.0,1347.0,Cygnal,NaN,NaN,NaN,NaN,2/4/25,2/5/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,46.2,47.2,NaN


In [5]:
approval.columns.values

array(['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'sponsors',
       'methodology', 'state', 'start_date', 'end_date',
       'sponsor_candidate_id', 'sponsor_candidate',
       'sponsor_candidate_party', 'question_id', 'sample_size',
       'population', 'subpopulation', 'population_full', 'tracking',
       'created_at', 'notes', 'url_article', 'url_topline',
       'url_crosstab', 'source', 'internal', 'partisan', 'race_id',
       'cycle', 'office_type', 'seat_number', 'seat_name',
       'election_date', 'stage', 'nationwide_batch', 'dem', 'rep', 'ind'],
      dtype=object)

In [6]:
morris_to_silver_dict = {
    'HarrisX/Harris Poll':'Harris Insights & Analytics',
    'Clarity': 'Clarity Campaign Labs',
    'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
    'Monmouth': 'Monmouth University',
    'HarrisX': 'Harris Insights & Analytics',
    'UMass Amherst/YouGov': 'YouGov',
    'Emerson': 'Emerson College',
    'Marquette Law School': 'Marquette University Law School',
    'NewsNation/Decision Desk HQ': 'NewsNation',
    'CNN/SSRS': 'SSRS',
    'Suffolk': 'Suffolk University',
    'Harris Poll': 'Harris Insights & Analytics',
    'Fabrizio/Impact': 'Fabrizio Lee & Associates/Impact Research',
    'Winston': 'The Winston Group',
    'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
    'Quinnipiac': 'Quinnipiac University'
}

In [7]:
approval['pollster'] = approval['pollster'].replace(morris_to_silver_dict)
approval = approval.rename({'population':'population_empty'}, axis='columns')
approval = approval.rename({'population_full':'population'}, axis='columns')
approval.head(7)

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,methodology,state,start_date,end_date,sponsor_candidate_id,...,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,dem,rep,ind
0,89624.0,1102.0,Emerson College,NaN,NaN,Online Opt-In Panel/Email,NaN,3/2/25,3/3/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,44.4,41.4,NaN
1,89604.0,1554.0,RMG Research,2178.0,Napolitan News Service,NaN,NaN,2/20/25,2/21/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,46.0,48.0,NaN
2,89579.0,568.0,YouGov,352.0,Economist,Online Matched Sample,NaN,2/16/25,2/18/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,35.0,37.0,NaN
3,89579.0,568.0,YouGov,352.0,Economist,Online Matched Sample,NaN,2/16/25,2/18/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,42.0,41.0,NaN
4,89590.0,294.0,McLaughlin,NaN,NaN,Online Opt-In Panel,NaN,2/11/25,2/18/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,41.0,48.0,NaN
5,89596.0,1497.0,co/efficient,NaN,NaN,NaN,NaN,2/15/25,2/17/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,44.0,46.0,NaN
6,89566.0,1302.0,Echelon Insights,NaN,NaN,Online Opt-In Panel,NaN,2/10/25,2/13/25,NaN,...,2026,U.S. House,NaN,Generic,11/3/26,general,False,46.0,47.0,NaN


In [8]:
approval.shape

(34, 37)

In [9]:
approval = approval[approval['partisan'].isna()]# [pd.to_datetime(approval['end_date']) >= pd.to_datetime('2023-12-01')]
approval.shape

(28, 37)

In [10]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [11]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [12]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(28, 37)

In [13]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='inner')
approval.shape

(23, 39)

In [14]:
approval = approval[(~approval['sample_size'].isna())]
approval.shape

(23, 39)

In [15]:
approval['pollster'].value_counts()

pollster
Cygnal                                                5
YouGov                                                4
Echelon Insights                                      3
Clarity Campaign Labs                                 3
RMG Research                                          2
Fabrizio Lee & Associates/Impact Research             2
Hart Research Associates/Public Opinion Strategies    2
Emerson College                                       1
Public Opinion Strategies                             1
Name: count, dtype: int64

In [16]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [17]:
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,2.00
1,D,1.02
2,D,0.63
3,D,0.79
4,R,0.60
...,...,...
93,R,2.44
94,R,0.45
95,R,3.23
96,R,4.94


In [18]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[0] * house_eff[1]
house_eff

,0,1,house_effect
0,-1,2.00,-2.00
1,1,1.02,1.02
2,1,0.63,0.63
3,1,0.79,0.79
4,-1,0.60,-0.60
...,...,...,...
93,-1,2.44,-2.44
94,-1,0.45,-0.45
95,-1,3.23,-3.23
96,-1,4.94,-4.94


In [19]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-2.00
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.02
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.63
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.79
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.60
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.44
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.45
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.23


In [20]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['dem_adj'] = df['dem'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['rep_adj'] = df['rep'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['dem_adj'] = df['dem']
        df['rep_adj'] = df['rep']
    return df

In [21]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,methodology,state,start_date,end_date,sponsor_candidate_id,...,election_date,stage,nationwide_batch,dem,rep,ind,Pollster,Predictive Plus-Minus,dem_adj,rep_adj
6,89560.0,1347.0,Cygnal,NaN,NaN,NaN,NaN,2/4/25,2/5/25,NaN,...,11/3/26,general,False,46.2,47.2,NaN,Cygnal,-0.747339,46.500,46.900
9,89491.0,1347.0,Cygnal,NaN,NaN,NaN,NaN,1/9/25,1/12/25,NaN,...,11/3/26,general,False,44.6,46.8,NaN,Cygnal,-0.747339,44.900,46.500
10,89437.0,1347.0,Cygnal,NaN,NaN,NaN,NaN,12/9/24,12/11/24,NaN,...,11/3/26,general,False,44.6,48.4,NaN,Cygnal,-0.747339,44.900,48.100
12,89655.0,NaN,Cygnal,NaN,NaN,NaN,NaN,3/3/25,3/5/25,NaN,...,11/3/26,general,NaN,47.4,45.9,NaN,Cygnal,-0.747339,47.700,45.600
17,89899.0,NaN,Cygnal,NaN,NaN,NaN,NaN,4/1/25,4/3/25,NaN,...,11/3/26,general,NaN,47.5,46.5,NaN,Cygnal,-0.747339,47.800,46.200
2,89579.0,568.0,YouGov,352.0,Economist,Online Matched Sample,NaN,2/16/25,2/18/25,NaN,...,11/3/26,general,False,35.0,37.0,NaN,YouGov,0.047635,34.750,37.250
3,89579.0,568.0,YouGov,352.0,Economist,Online Matched Sample,NaN,2/16/25,2/18/25,NaN,...,11/3/26,general,False,42.0,41.0,NaN,YouGov,0.047635,41.750,41.250
18,89901.0,NaN,YouGov,NaN,Economist,Online Matched Sample,NaN,4/5/25,4/8/25,NaN,...,11/3/26,general,NaN,39.0,36.0,NaN,YouGov,0.047635,38.750,36.250
19,89901.0,NaN,YouGov,NaN,Economist,Online Matched Sample,NaN,4/5/25,4/8/25,NaN,...,11/3/26,general,NaN,43.0,42.0,NaN,YouGov,0.047635,42.750,42.250
4,89566.0,1302.0,Echelon Insights,NaN,NaN,Online Opt-In Panel,NaN,2/10/25,2/13/25,NaN,...,11/3/26,general,False,46.0,47.0,NaN,Echelon Insights,-0.216672,46.000,47.000


In [22]:
approval = approval_adj.copy()
approval.shape

(23, 41)

In [23]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id')]
    ndup = data[~data.duplicated(subset='poll_id')]
    #try:
    dup_a = dup[dup['population'].isin(['a'])]
#     except ValueError:
#         print(dup['population'].isin(['a']))
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id')]
    ndup = df[~df.duplicated(subset='poll_id')]
    dup_lv = dup[dup['population'].isin(['rv'])]
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [24]:
approval = approval.rename({'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [25]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs',
       'Trafalgar Group', 'OnMessage Inc.', 'McLaughlin & Associates',
       'North Star Opinion Research'], dtype=object)

In [26]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,methodology,state,start_date,end_date,sponsor_candidate_id,...,election_date,stage,nationwide_batch,dem,rep,ind,Pollster,pred_plus_minus,dem_adj,rep_adj
6,89560.0,1347.0,Cygnal,NaN,NaN,NaN,NaN,2/4/25,2/5/25,NaN,...,11/3/26,general,False,46.2,47.2,NaN,Cygnal,-0.747339,46.5,46.9
9,89491.0,1347.0,Cygnal,NaN,NaN,NaN,NaN,1/9/25,1/12/25,NaN,...,11/3/26,general,False,44.6,46.8,NaN,Cygnal,-0.747339,44.9,46.5
10,89437.0,1347.0,Cygnal,NaN,NaN,NaN,NaN,12/9/24,12/11/24,NaN,...,11/3/26,general,False,44.6,48.4,NaN,Cygnal,-0.747339,44.9,48.1
12,89655.0,NaN,Cygnal,NaN,NaN,NaN,NaN,3/3/25,3/5/25,NaN,...,11/3/26,general,NaN,47.4,45.9,NaN,Cygnal,-0.747339,47.7,45.6
17,89899.0,NaN,Cygnal,NaN,NaN,NaN,NaN,4/1/25,4/3/25,NaN,...,11/3/26,general,NaN,47.5,46.5,NaN,Cygnal,-0.747339,47.8,46.2


In [27]:
approval.shape

(23, 41)

In [28]:
# net: negative --> dem advantage, positive --> rep advantage
approval['net'] = approval['rep'] - approval['dem']
approval.head(1)

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,methodology,state,start_date,end_date,sponsor_candidate_id,...,stage,nationwide_batch,dem,rep,ind,Pollster,pred_plus_minus,dem_adj,rep_adj,net
6,89560.0,1347.0,Cygnal,NaN,NaN,NaN,NaN,2/4/25,2/5/25,NaN,...,general,False,46.2,47.2,NaN,Cygnal,-0.747339,46.5,46.9,1.0


In [29]:
approval['url'] = approval['url_crosstab'].fillna(approval['url_topline']).fillna(approval['url_article'].fillna(''))

In [30]:
approval_polls = approval[['pollster', 'start_date', 'end_date', 'sample_size', 'population', 'dem', 'rep', 'dem_adj', 'rep_adj', 'net', 'pred_plus_minus', 'poll_id', 'url', 'sponsors']]
approval_polls

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,pred_plus_minus,poll_id,url,sponsors
6,Cygnal,2/4/25,2/5/25,1500.0,lv,46.2,47.2,46.500,46.900,1.0,-0.747339,89560.0,https://www.cygn.al/wp-content/uploads/2025/02...,NaN
9,Cygnal,1/9/25,1/12/25,1500.0,lv,44.6,46.8,44.900,46.500,2.2,-0.747339,89491.0,https://www.cygn.al/wp-content/uploads/2025/01...,NaN
10,Cygnal,12/9/24,12/11/24,1500.0,lv,44.6,48.4,44.900,48.100,3.8,-0.747339,89437.0,https://www.cygn.al/wp-content/uploads/2024/12...,NaN
12,Cygnal,3/3/25,3/5/25,1500.0,lv,47.4,45.9,47.700,45.600,-1.5,-0.747339,89655.0,https://www.cygn.al/wp-content/uploads/2025/03...,NaN
17,Cygnal,4/1/25,4/3/25,1500.0,lv,47.5,46.5,47.800,46.200,-1.0,-0.747339,89899.0,https://www.cygn.al/wp-content/uploads/2025/04...,NaN
2,YouGov,2/16/25,2/18/25,1603.0,a,35.0,37.0,34.750,37.250,2.0,0.047635,89579.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,Economist
3,YouGov,2/16/25,2/18/25,1451.0,rv,42.0,41.0,41.750,41.250,-1.0,0.047635,89579.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,Economist
18,YouGov,4/5/25,4/8/25,1741.0,a,39.0,36.0,38.750,36.250,-3.0,0.047635,89901.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,Economist
19,YouGov,4/5/25,4/8/25,1558.0,rv,43.0,42.0,42.750,42.250,-1.0,0.047635,89901.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,Economist
4,Echelon Insights,2/10/25,2/13/25,1010.0,lv,46.0,47.0,46.000,47.000,1.0,-0.216672,89566.0,https://view.officeapps.live.com/op/view.aspx?...,NaN


In [31]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    # state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    # linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    linear_weights = delta.map(lambda x: (1 - x/60) if x < 60 else 0)
    exp_weights = 0.1**(delta/(45))
    state_pivot['time_weights'] =  0.1 * linear_weights + 0.9 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'rv':
            return 0.8
        elif population == 'lv':
            return 0.6
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Gather the weights together
    state_pivot['total_weights'] = state_pivot['sample_size_weights'] * state_pivot['time_weights'] * state_pivot['quality_weights'] * state_pivot['population_weights']
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [32]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,pred_plus_minus,poll_id,url,sponsors,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
6,Cygnal,2025-02-04,2025-02-05,1500.0,lv,46.2,47.2,46.500,46.900,1.0,-0.747339,89560.0,https://www.cygn.al/wp-content/uploads/2025/02...,NaN,0.051561,0.021479,1.776911,0.6,0.006407
9,Cygnal,2025-01-09,2025-01-12,1500.0,lv,44.6,46.8,44.900,46.500,2.2,-0.747339,89491.0,https://www.cygn.al/wp-content/uploads/2025/01...,NaN,0.051561,0.006291,1.776911,0.6,0.001876
10,Cygnal,2024-12-09,2024-12-11,1500.0,lv,44.6,48.4,44.900,48.100,3.8,-0.747339,89437.0,https://www.cygn.al/wp-content/uploads/2024/12...,NaN,0.051561,0.001223,1.776911,0.6,0.000365
12,Cygnal,2025-03-03,2025-03-05,1500.0,lv,47.4,45.9,47.700,45.600,-1.5,-0.747339,89655.0,https://www.cygn.al/wp-content/uploads/2025/03...,NaN,0.051561,0.115000,1.776911,0.6,0.034303
17,Cygnal,2025-04-01,2025-04-03,1500.0,lv,47.5,46.5,47.800,46.200,-1.0,-0.747339,89899.0,https://www.cygn.al/wp-content/uploads/2025/04...,NaN,0.051561,0.470239,1.776911,0.6,0.140267
2,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.750,37.250,2.0,0.047635,89579.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,Economist,0.053302,0.041774,0.964021,1.0,0.011648
18,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.750,36.250,-3.0,0.047635,89901.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,Economist,0.055549,0.594290,0.964021,1.0,0.172686
4,Echelon Insights,2025-02-10,2025-02-13,1010.0,lv,46.0,47.0,46.000,47.000,1.0,-0.216672,89566.0,https://view.officeapps.live.com/op/view.aspx?...,NaN,0.042310,0.032344,1.181365,0.6,0.005263
13,Echelon Insights,2025-03-10,2025-03-13,1007.0,lv,46.0,47.0,46.000,47.000,1.0,-0.216672,89663.0,https://echelonin.wpenginepowered.com/wp-conte...,NaN,0.042247,0.173859,1.181365,0.6,0.028250
20,Echelon Insights,2025-04-10,2025-04-14,1014.0,lv,47.0,47.0,47.000,47.000,0.0,-0.216672,89949.0,https://echelonin.wpenginepowered.com/wp-conte...,NaN,0.042393,0.788504,1.181365,0.6,0.128568


In [33]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 24), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['rep_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['dem_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['rep_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['dem_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Republicans':yes_averages, 'Democrats':no_averages}), pd.DataFrame({'Date':date_range, 'Republicans':yes_stdevs, 'Democrats':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [34]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Republicans,Democrats
0,2025-01-24,48.193974,44.581888
1,2025-01-25,48.193839,44.581665
2,2025-01-26,48.193731,44.581404
3,2025-01-27,48.193652,44.581102
4,2025-01-28,48.193606,44.580754
...,...,...,...
81,2025-04-15,44.054370,45.176655
82,2025-04-16,44.055830,45.179491
83,2025-04-17,44.057444,45.182560
84,2025-04-18,44.059224,45.185879


In [35]:
polls_df.shape

(21, 19)

In [36]:
display_df = polls_df[['pollster', 'start_date', 'end_date', 'sample_size', 'population', 'dem', 'rep', 'net',
                      'total_weights', 'url', 'sponsors']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 'total_weights':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population'].map(lambda x: x.upper()).astype(str)
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net
0,"<a href=""https://echelonin.wpenginepowered.com...",2025-04-10,2025-04-14,1014 LV,0.128568,47.0,47.0,0.0
1,"<a href=""https://45840940.fs1.hubspotuserconte...",2025-04-09,2025-04-14,1008 RV,0.121337,44.0,45.0,1.0
2,"<a href=""https://www.cnbc.com/video/2025/04/17...",2025-04-09,2025-04-13,1000 A,0.208385,48.0,46.0,-2.0
3,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-04-05,2025-04-08,1741 A,0.172686,39.0,36.0,-3.0
4,"<a href=""https://www.cygn.al/wp-content/upload...",2025-04-01,2025-04-03,1500 LV,0.140267,47.5,46.5,-1.0
5,"<a href=""https://prod-i.a.dj.com/public/resour...",2025-03-27,2025-04-01,1500 RV,0.019173,44.0,43.0,-1.0
6,"<a href=""https://echelonin.wpenginepowered.com...",2025-03-10,2025-03-13,1007 LV,0.028250,46.0,47.0,1.0
7,"<a href=""https://s3.documentcloud.org/document...",2025-03-07,2025-03-11,1000 RV,0.034877,48.0,47.0,-1.0
8,"<a href=""https://static1.squarespace.com/stati...",2025-03-03,2025-03-09,1036 RV,0.022170,46.0,44.0,-2.0
9,"<a href=""https://www.cygn.al/wp-content/upload...",2025-03-03,2025-03-05,1500 LV,0.034303,47.4,45.9,-1.5


In [37]:
display_df.to_csv('generic_ballot_polls_display_table.csv')

In [38]:
approval_avg['Net'] = approval_avg['Republicans'] - approval_avg['Democrats']
approval_avg

,Date,Republicans,Democrats,Net
0,2025-01-24,48.193974,44.581888,3.612086
1,2025-01-25,48.193839,44.581665,3.612174
2,2025-01-26,48.193731,44.581404,3.612327
3,2025-01-27,48.193652,44.581102,3.612550
4,2025-01-28,48.193606,44.580754,3.612851
...,...,...,...,...
81,2025-04-15,44.054370,45.176655,-1.122285
82,2025-04-16,44.055830,45.179491,-1.123661
83,2025-04-17,44.057444,45.182560,-1.125116
84,2025-04-18,44.059224,45.185879,-1.126655


In [39]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Republicans_x'] + 1.96*approval_trendline['Republicans_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Republicans_x'] - 1.96*approval_trendline['Republicans_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Democrats_x'] + 1.96*approval_trendline['Democrats_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Democrats_x'] - 1.96*approval_trendline['Democrats_y']
approval_trendline = approval_trendline.rename({
    'Republicans_x':'approve_mean',
    'Democrats_x':'disapprove_mean',
}, axis='columns').drop(['Republicans_y', 'Democrats_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-24,48.193974,44.581888,52.259591,44.128357,45.471008,43.692767
1,2025-01-25,48.193839,44.581665,52.261045,44.126634,45.470944,43.692387
2,2025-01-26,48.193731,44.581404,52.262756,44.124707,45.470868,43.691941
3,2025-01-27,48.193652,44.581102,52.264745,44.122559,45.470779,43.691424
4,2025-01-28,48.193606,44.580754,52.267037,44.120174,45.470677,43.690832
...,...,...,...,...,...,...,...
81,2025-04-15,44.054370,45.176655,51.687333,36.421406,52.003848,38.349461
82,2025-04-16,44.055830,45.179491,51.687950,36.423710,51.999379,38.359603
83,2025-04-17,44.057444,45.182560,51.688695,36.426192,51.994468,38.370652
84,2025-04-18,44.059224,45.185879,51.689582,36.428866,51.989080,38.382678


In [40]:
# remember: approve = Republicans, disapprove = Democrats - kept for DataWrapper setup simplicity
# Export generic ballot averages
approval_trendline.to_csv('generic_ballot_stats.csv')

In [41]:
# line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# # scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
# line.update_traces(hovertemplate=None)
# line.update_layout(hovermode='x unified')
# # scatter.update_traces(hovertemplate=None, hoverinfo='skip')
# fig_approve_CI = go.Figure([
#     go.Scatter(
#         name='Approve CI Upper Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
#         mode='lines',
#         marker=dict(color='#8be8f2'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),
#     go.Scatter(
#         name='Approve CI Lower Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
#         mode='lines',
#         marker=dict(color='#8be8f2'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),  
# ])
# fig_disapprove_CI = go.Figure([
#     go.Scatter(
#         name='Disapprove CI Upper Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
#         mode='lines',
#         marker=dict(color='#a3cbec'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),
#     go.Scatter(
#         name='Disapprove CI Lower Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
#         mode='lines',
#         marker=dict(color='#a3cbec'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),  
# ])

# fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
# fig.update_layout(
#     title='Congressional Approval Rating',
#     xaxis_title='Date',
#     yaxis_title='%',
#     legend_title='Legend',
#     hovermode='x unified'
# )
# fig.show()